# Determining Best Deal, Worst Deal By Predicating Offer Pricing 

**Introduction**

In this post, I'm going to walk through designing a pricing prediction by training model using 3 machine learning algorithms Multiple linear regression, Random Forest regression, and XGBoost to predict the price.

This technique can be used to solve any valuation prediction problem and can be applied to multiple business use cases like calculating the pricing deal by predicting pricing and comparing the offered price against predicated price.

In order to determine if the offer price is good deal or not we need to solve problem of predicting offer price and then comparing it with Seller's offer price. In the current example, we can use key vehicle features that contribute to the used car prices to make the price prediction.

To solve this problem without any background in machine learning or data science, analytics might be a bit of a stretch, but I took a stab at it using the available information.

**Task #1 - IMPORT DATASETS and LIBRARIES**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installedhttps://github.com/satyajitnalavade/price-prediction/blob/main/cars_data.csv
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install xgboost

In [ ]:
!pip install wordcloud

In [ ]:
!pip install plotly

In [ ]:
import numpy as np # Multi-dimensional array object
import pandas as pd # Data Manipulation
import seaborn as sns # Data Visualization
import matplotlib.pyplot as plt # Data Visualization
import plotly.express as px # Interactive Data Visualization
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot # Offline version of the Plotly modules.

In [ ]:
# Read the CSV file 
car_df = pd.read_csv("/kaggle/input/cars1/CARS.csv")

**Task #2 - DATASETS EXPLORARTION**

In [ ]:
# Load the top 10 instances
car_df.head(10)

In [ ]:
# Load the bottom 10 instances 
car_df.tail(10)

In [ ]:
car_df.columns

In [ ]:
car_df.shape

* **Task #3 - DATA SANITIZATION**

In [ ]:
# Check for null column values
car_df.isnull().sum()

In [ ]:
# Drop 2 row with null values
car_df = car_df.dropna()


In [ ]:
# Check for null column values
car_df.isnull().sum()

In [ ]:
# Obtain the summary of the dataframe
car_df.info()

In [ ]:
# Convert MSRP and Invoice datatype to integer so we need to remove $ sign and comma (,) from these 2 columns

car_df["MSRP"] = car_df["MSRP"].str.replace("$", "")
car_df["MSRP"] = car_df["MSRP"].str.replace(",", "")
car_df["MSRP"] = car_df["MSRP"].astype(int)

In [ ]:
car_df["Invoice"] = car_df["Invoice"].str.replace("$", "")
car_df["Invoice"] = car_df["Invoice"].str.replace(",", "")
car_df["Invoice"] = car_df["Invoice"].astype(int)

In [ ]:
# Let's view the updated MSRP and Invoice Columns
car_df.head()

In [ ]:
# Display the updated summary of the dataframe
car_df.info()

In [ ]:
# Let's view various makes of the cars
car_df.Make = car_df.Make.str.lower()
car_df.Make.unique()

In [ ]:
# Seems there is no invalid value to fix in Make

* **Task #4 - DATA VISUALIZATION**

In [ ]:
# Plot Make
fig = px.histogram(car_df, x = "Make",
                  labels = {"Make":"Manufacturer"},
                  title = "MAKE OF THE CAR",
                  color_discrete_sequence = ["maroon"])
                  
fig.show()

In [ ]:
# Let's view various types of the cars
car_df.Type.unique()

In [ ]:
fig = px.histogram(car_df, x = "Type",
                  labels = {"Type":"Type"},
                  title = "TYPE OF THE CAR",
                  color_discrete_sequence = ["blue"])
                  
fig.show()

In [ ]:
# Let's plot the location
car_df.Origin.unique()

In [ ]:
# Let's plot the location
fig = px.histogram(car_df, x = "Origin",
                  labels = {"Origin":"Origin"},
                  title = "LOCATION OF THE CAR SALES",
                  color_discrete_sequence = ["red"])
                  
fig.show()

In [ ]:
# Let's view the drivetrain of the cars
car_df.DriveTrain.unique()

In [ ]:
fig = px.histogram(car_df, x = "DriveTrain",
                  labels = {"DriveTrain":"Drivetrain"},
                  title = "DRIVETRAIN OF THE CAR",
                  color_discrete_sequence = ["BLACK"])
                  
fig.show()

In [ ]:
# Plot the make of the car and its location
fig = px.histogram(car_df, x = "Make",
                  color = "Origin",
                  labels = {"Make":"Manufacturer"},
                  title = "MAKE OF THE CAR Vs LOCATION")
                  
fig.show()

In [ ]:
# Plot the make of the car and type
fig = px.histogram(car_df, x = "Make",
                  color = "Type",
                  labels = {"Make":"Manufacturer"},
                  title = "MAKE OF THE CAR Vs TYPE")
                  
fig.show()

In [ ]:
#Checking for duplicates
car_df.loc[car_df.duplicated()]

In [ ]:
sns.pairplot(data = car_df)
# scatterplots for joint relationships and histograms for univariate distributions

In [ ]:
car_df

In [ ]:
# Let's view the model of all used cars using WordCloud generator
from wordcloud import WordCloud, STOPWORDS
text = car_df.Model.values
stopwords = set(STOPWORDS)

wc = WordCloud(background_color = "black", max_words = 2000, max_font_size = 100, random_state = 3, 
              stopwords = stopwords, contour_width = 3).generate(str(text)) 

In [ ]:
fig = plt.figure(figsize = (25, 15))
plt.imshow(wc, interpolation = "bilinear")
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize=(25,8))

plt.subplot(1,2,1)
plt.title('Vehicle Price Distribution')
sns.distplot(car_df.MSRP)

plt.subplot(1,2,2)
plt.title('Vehicle Price Spread')
sns.boxplot(y=car_df.MSRP)

plt.show()

In [ ]:
print(car_df.MSRP.describe(percentiles = [0.25,0.50,0.75,0.85,0.90,1]))

In [ ]:
# Obtain the correlation matrix
car_df.corr()

In [ ]:
plt.figure(figsize = (18,18))
sns.heatmap(car_df.corr(), annot = True)

In [ ]:
car_df.head()

In [ ]:
# Perform One-Hot Encoding for "Make", "Model", "Type", "Origin", and "DriveTrain"
df_dum = pd.get_dummies(car_df, columns=["Make", "Model", "Type", "Origin", "DriveTrain"])

In [ ]:
df_dum.head()

In [ ]:
# Invoice feature do not contribute to car price prediction 
df_data = df_dum.drop(["Invoice"], axis = 1)

In [ ]:
df_data.info()

In [ ]:
df_data.columns

In [ ]:
df_data.shape

In [ ]:
# Feeding input features to X and output (MSRP) to y
X = df_data.drop("MSRP", axis = 1)
y = df_data["MSRP"]

In [ ]:
X

In [ ]:
y

**Task #5 - PREPARE TRAINING AND TEST DATA**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 90-10 Training and Test Split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

**Task #6 TRAIN AND EVALUATE A MULTIPLE LINEAR REGRESSION**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from math import sqrt

In [ ]:
LinearRegression_model= LinearRegression()
LinearRegression_model.fit(X_train, y_train)

In [ ]:
accuracy_LinearRegression = LinearRegression_model.score(X_test, y_test)
accuracy_LinearRegression

**Task #7 TRAIN AND EVALUATE A DECISION TREE MODEL**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DecisionTree_model = DecisionTreeRegressor()
DecisionTree_model.fit(X_train, y_train)

In [ ]:
accuracy_DecisionTree = DecisionTree_model.score(X_test, y_test)
accuracy_DecisionTree

**Task #8 TRAIN AND EVALUATE A RANDOM FOREST MODEL**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RandomForest_model = RandomForestRegressor(n_estimators= 10, max_depth= 5)
RandomForest_model.fit(X_train, y_train)

In [ ]:
accuracy_RandomForest= RandomForest_model.score(X_test, y_test)
accuracy_RandomForest

**Task #9 TRAIN AN XG-BOOST REGRESSOR MODEL**

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)

In [ ]:
accuracy_XGBoost = model.score(X_test, y_test)
accuracy_XGBoost

**Task #10 COMPARE MODELS AND CALCULATE REGRESSION KPIs**

In [ ]:
y_predict_linear = LinearRegression_model.predict(X_test)

fig = sns.regplot(y_predict_linear, y_test, color = 'red', marker = "^")
fig.set(title = "Linear Regression Model", xlabel = "Predicted Price of the used cars ($)", ylabel = "Actual Price of the used cars ($)")

In [ ]:
RMSE= float(format(np.sqrt(mean_squared_error(y_test, y_predict_linear)), ".3f"))
MSE= mean_squared_error(y_test, y_predict_linear)
MAE= mean_absolute_error(y_test, y_predict_linear)
r2= r2_score(y_test, y_predict_linear)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2)

In [ ]:
y_predict_RandomForest = RandomForest_model.predict(X_test)

fig = sns.regplot(y_predict_RandomForest, y_test, color = 'blue', marker = "s")
fig.set(title = "Random Forest Regression Model", xlabel = "Predicted Price of the used cars ($)", ylabel= "Actual Price of the used cars ($)")

In [ ]:
RMSE= float(format(np.sqrt(mean_squared_error(y_test, y_predict_RandomForest)), ".3f"))
MSE= mean_squared_error(y_test, y_predict_RandomForest)
MAE= mean_absolute_error(y_test, y_predict_RandomForest)
r2= r2_score(y_test, y_predict_RandomForest)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2) 

In [ ]:
y_predict_XGBoost = model.predict(X_test)

fig = sns.regplot(y_predict_XGBoost, y_test, color = 'green', marker = "D")
fig.set(title = "XGBoost Model", xlabel = "Predicted Price of the used cars ($)", ylabel = "Actual Price of the used cars ($)")

In [ ]:
RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict_XGBoost)), ".3f"))
MSE = mean_squared_error(y_test, y_predict_XGBoost)
MAE = mean_absolute_error(y_test, y_predict_XGBoost)
r2 = r2_score(y_test, y_predict_XGBoost)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2)

# **Conclusion**

From the ablove results, it is clearly shown that XGBoost model scores 96% accuracy which outperforms Linear Regression and Random Forest Regression models
Using Prediction we can calculate the Vehicle Pricing and use it to determine if the Seller Offer price is good Deal or not a deal